In [36]:
import requests
from zipfile import ZipFile
from io import BytesIO

from datetime import timedelta
from datetime import datetime
import pandas as pd

from airflow import DAG
from airflow.operators.python import PythonOperator # Так как мы пишет такси в питоне

TOP_1M_DOMAINS = 'http://s3.amazonaws.com/alexa-static/top-1m.csv.zip'
TOP_1M_DOMAINS_FILE = 'top-1m.csv'


def get_data():
    top_doms = requests.get(TOP_1M_DOMAINS, stream=True)
    zipfile = ZipFile(BytesIO(top_doms.content))
    top_data = zipfile.read(TOP_1M_DOMAINS_FILE).decode('utf-8')
    
    with open(TOP_1M_DOMAINS_FILE, 'w') as f:
        f.write(top_data)


def get_stats():
    top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    top_data_top_10  = top_data_df[top_data_df.domain.str.endswith('.ru')][:10]
    with open('top_data_top_10.csv', 'w') as f:
        f.write(top_data_top_10.to_csv(index=False, header=False))


def print_data():
    with open('top_data_top_10.csv', 'r') as f:
        all_data = f.read()
    print('Top domains in .Ru')
    print(all_data)
    with open('report.txt', 'w') as f:
        f.write(f'Top domains in .Ru \n{datetime.today()}\n')
        f.write(all_data)

default_args = {
    'owner': 'your_name', # Владелец операции 
    'depends_on_past': False, # Зависимость от прошлых запусков

    'schedule_interval': '0 12 * * *', # cron выражение, также можно использовать '@daily', '@weekly'
    #'schedule_interval': '@daily' переменные airflow
    #'schedule_interval': timedelta() параметр timedelta

    'retries': 2, # Кол-во попыток выполнить DAG
    'retry_delay': timedelta(minutes=5), # Промежуток между перезапусками

    'email': '', # Почта для уведомлений 
    'email_on_failure': '', # Почта для уведомлений при ошибке
    'email_on_retry': '', # Почта для уведомлений при перезапуске

    'retry_exponential_backoff': '', # Для установления экспоненциального времени между перезапусками
    'max_retry_delay': '', # Максимальный промежуток времени для перезапуска

    'start_date': datetime(2021, 10, 7), # Дата начала выполнения DAG
    'end_date': '', # Дата завершения выполнения DAG

    'on_failure_callback': '', # Запустить функцию если DAG упал
    'on_success_callback': '', # Запустить функцию если DAG выполнился
    'on_retry_callback': '', # Запустить функцию если DAG ушел на повторный запуск
    'on_execute_callback': '', # Запустить функцию если DAG начал выполняться
     # Задать документацию
    'doc': '',
    'doc_md': '',
    'doc_rst': '',
    'doc_json': '',
    'doc_yaml': ''
}

dag = DAG('DAG_name', default_args=default_args)

t1 = PythonOperator(task_id = 'get_data',
            python_callable= get_data, dag = dag)

t2 = PythonOperator(task_id = 'get_stats',
            python_callable = get_stats, dag = dag)

t3 = PythonOperator(task_id = 'print_data',
            python_callable = print_data, dag = dag)


t1 >> t2 >> t3

#t1.set_downstream(t2)
#t2.set_downstream(t3)


<Task(PythonOperator): print_data>

In [37]:
print_data()

Top domains in .Ru
30,yandex.ru
43,dzen.ru
46,mail.ru
93,ok.ru
110,avito.ru
167,ya.ru
173,google.ru
181,gosuslugi.ru
196,ozon.ru
254,wildberries.ru



In [25]:
today = datetime.today()

# dd/mm/YY
d1 = today
print("d1 =", d1)

d1 = 2022-09-16 22:51:32.530101


In [30]:
curr_date = datetime.today().strftime('%Y-%m-%d')

In [31]:
curr_date

'2022-09-16'

In [4]:
get_data(),
get_stats()

In [10]:
pd.read_csv('top_data_top_10.csv')

,30,yandex.ru
0,43,dzen.ru
1,46,mail.ru
2,93,ok.ru
3,110,avito.ru
4,167,ya.ru
5,173,google.ru
6,181,gosuslugi.ru
7,196,ozon.ru
8,254,wildberries.ru


In [11]:
print_data()

Top domains in .Ru
30,yandex.ru
43,dzen.ru
46,mail.ru
93,ok.ru
110,avito.ru
167,ya.ru
173,google.ru
181,gosuslugi.ru
196,ozon.ru
254,wildberries.ru



In [12]:
timedelta(minutes=5)

datetime.timedelta(seconds=300)

In [3]:
datetime(2021, 9, 16)

datetime.datetime(2021, 9, 16, 0, 0)

In [7]:
datetime(2022, 9, 16)

datetime.datetime(2022, 9, 16, 0, 0)

In [5]:
import os
print(os.getcwd())

/mnt/HC_Volume_18315164/home-jupyter/jupyter-m-pleshakov-24/airflow_rep/airflow/dags/m-pleshakov-24


In [ ]:
Необходимо выполнить:

1) Поменять имена dag на уникальные (лучше всего как-то использовать свой логин).

Поставить новую дату начала DAG и новый интервал (все еще должен быть ежедневным)

2) Удалить таски get_stat и get_stat_com. Вместо них сделать свои собственные, которые

считают следующие:

Найти топ-10 доменных зон по численности доменов
Найти домен с самым длинным именем (если их несколько, то взять только первый в алфавитном порядке)
На каком месте находится домен airflow.com?
3) Финальный таск должен писать в лог результат ответы на вопросы выше

In [317]:
import requests
import pandas as pd
from datetime import timedelta
from datetime import datetime
import os.path

curr_date = datetime.today().strftime("%d-%m-%y %H:%M")

from airflow import DAG
from airflow.operators.python import PythonOperator

TOP_1M_DOMAINS = 'http://s3.amazonaws.com/alexa-static/top-1m.csv.zip'
TOP_1M_DOMAINS_FILE = 'top-1m.csv'


def get_data():
    # Здесь пока оставили запись в файл, как передавать переменую между тасками будет в третьем уроке
    top_doms = pd.read_csv(TOP_1M_DOMAINS)
    top_data = top_doms.to_csv(index=False)
    with open(TOP_1M_DOMAINS_FILE, 'w') as f:
        f.write(top_data)
        
#Найти топ-10 доменных зон по численности доменов
def get_top10():
    top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    top_data_df = top_data_df['domain'].str.split('.').apply(lambda x: x[-1]).value_counts()[:10]
    top_data_df = pd.DataFrame(top_data_df).reset_index().rename(columns={'domain':'counts','index':'domain'})
    return 'Top 10 domains by regions: \n' + str(top_data_df) + '\n'

#Найти домен с самым длинным именем (если их несколько, то взять только первый в алфавитном порядке)
def get_max_length():
    top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    max_length = top_data_df.domain.str.len().max()
    return 'Largest domain: \n' + str(list(top_data_df[top_data_df.domain.str.len() == max_length]
                                                             .domain.sort_values().values)[0])+'\n'

#На каком месте находится домен airflow.com?
def get_airflow_place():
    top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    mask =  top_data_df.domain.apply(lambda x : True if re.search('^airflow.com', x , re.IGNORECASE) else False)
    top_data_df = top_data_df[mask]
    lines = ''
    if len(top_data_df) > 0:
        lines = str(top_data_df)+'\n'
    else:
        lines = 'airflow.com not in top \n'
    return lines
    
def final_print(ds = 'none'):
    curr_date = ds
    lines = 'Report '+ curr_date + '\n'
    lines = lines + str(get_top10())+'\n'
    lines = lines + str(get_max_length())+'\n'
    lines = lines + str(get_airflow_place())+'\n'
    
    output_hist = ''
    if os.path.isfile('log-m-pleshakov.txt'):
        with open('log-m-pleshakov.txt', 'r') as f:
            output_hist = f.readlines()

    with open('log-m-pleshakov.txt', 'w') as f:
        for i in range(len(output_hist)):
            f.write(str(output_hist[i]))
        f.write(str(lines))
    print(lines)


default_args = {
    'owner': 'm.pleshakov',
    'depends_on_past': False,
    'retries': 2,
    'retry_delay': timedelta(minutes=3),
    'start_date': datetime(2021, 9, 17),
}
schedule_interval = '*/30 */1 * * *'

dag = DAG('project_less2_m-pleshakov', default_args=default_args, schedule_interval=schedule_interval)

t1 = PythonOperator(task_id='get_data',
                    python_callable=get_data,
                    dag=dag)

t2 = PythonOperator(task_id='get_top10',
                    python_callable=get_top10,
                    dag=dag)

t3 = PythonOperator(task_id='get_max_length',
                        python_callable=get_max_length,
                        dag=dag)

t4 = PythonOperator(task_id='get_airflow_place',
                        python_callable=get_airflow_place,
                        dag=dag)

t5 = PythonOperator(task_id='final_print',
                    python_callable=final_print,
                    dag=dag)

t1 >> [t2, t3, t4] >> t5

<Task(PythonOperator): final_print>

In [313]:
#Найти топ-10 доменных зон по численности доменов
def get_top10():
    top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    top_data_df = top_data_df['domain'].str.split('.').apply(lambda x: x[-1]).value_counts()[:10]
    top_data_df = pd.DataFrame(top_data_df).reset_index().rename(columns={'domain':'counts','index':'domain'})
    
    return 'Top 10 domains by regions: \n' + str(top_data_df) + '\n'

In [247]:
print(get_top10())

top 10 domains by regions 
com    361325
org     30508
net     25906
ru      20065
au      11726
de      10507
uk       9954
nl       9455
it       8846
in       8546
Name: domain, dtype: int64



In [256]:
#Найти домен с самым длинным именем (если их несколько, то взять только первый в алфавитном порядке)
def get_max_length():
    top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    max_length = top_data_df.domain.str.len().max()
    return 'Largest domain: \n' + str(list(top_data_df[top_data_df.domain.str.len() == max_length]
         .domain.sort_values().values)[0])+'\n'

In [249]:
print(get_max_length())

largest domain 
file-service-default-114c67af0763a8a98e770ff3ee495371.fra1.digitaloceanspaces.com



In [258]:
#На каком месте находится домен airflow.com?
def get_airflow_place():
    top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    mask =  top_data_df.domain.apply(lambda x : True if re.search('^airflow.com', x , re.IGNORECASE) else False)
    top_data_df = top_data_df[mask]
    lines = ''
    if len(top_data_df) > 0:
        lines = str(top_data_df)+'\n'
    else:
        lines = 'airflow.com not in top \n'
    return lines
    

In [252]:
print(get_airflow_place())

airflow.com not in top 17-09-22 20:35



In [305]:
def final_print():
    curr_date = datetime.today().strftime("%d-%m-%y %H:%M")
    lines = 'Report '+ curr_date + '\n'
    lines = lines + str(get_top10())+'\n'
    lines = lines + str(get_max_length())+'\n'
    lines = lines + str(get_airflow_place())+'\n'
    
    output_hist = ''
    if os.path.isfile('log-m-pleshakov.txt'):
        with open('log-m-pleshakov.txt', 'r') as f:
            output_hist = f.readlines()
            
    with open('log-m-pleshakov.txt', 'w') as f:
        for i in range(len(output_hist)):
            f.write(str(output_hist[i]))
        f.write(str(lines))

In [314]:
final_print()

Report 00:00
Top 10 domains by regions: 
  domain  counts
0    com  361325
1    org   30508
2    net   25906
3     ru   20065
4     au   11726
5     de   10507
6     uk    9954
7     nl    9455
8     it    8846
9     in    8546

Largest domain: 
file-service-default-114c67af0763a8a98e770ff3ee495371.fra1.digitaloceanspaces.com

airflow.com not in top 




NameError: name 'curr_time' is not defined

In [280]:
# with open('output.txt', 'w') as f:
#     f.write(str(top_data_df[:10]))
    
top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
top_data_df = top_data_df['domain'].str.split('.').apply(lambda x: x[-1]).value_counts()[:10]
top_data_df = pd.DataFrame(top_data_df).reset_index().rename(columns={'domain':'counts','index':'domain'})
    

,domain,counts
0,com,361325
1,org,30508
2,net,25906
3,ru,20065
4,au,11726
5,de,10507
6,uk,9954
7,nl,9455
8,it,8846
9,in,8546


In [264]:
top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])

In [156]:
datetime.datetime.today().strftime("%d-%m-%y %H:%M")

'17-09-22 20:26'

In [130]:
len(top_data_df)

0

In [79]:
top_data_df[top_data_df['domain'].values == 'google.com']

,rank,domain
0,1,google.com


In [78]:
top_data_df

,rank,domain
0,1,google.com
1,2,youtube.com
2,3,facebook.com
3,4,baidu.com
4,5,bilibili.com
...,...,...
705769,705770,zimbabweyp.com
705770,705771,zimhealthcsos.ngo
705771,705772,zimjobs.com
705772,705773,zimnews24.com


In [122]:
mask =  top_data_df.domain.apply(lambda x : True if re.search('^airflow.com', x , re.IGNORECASE) else False)
top_data_df[mask]

,rank,domain,domain_name


In [108]:
pd.Series('airflow.com').isin(top_data_df.domain.values)

0    False
dtype: bool

In [102]:
top_data_df.domain.values

array(['google.com', 'youtube.com', 'facebook.com', ..., 'zimjobs.com',
       'zimnews24.com', 'zwnews.com'], dtype=object)

In [109]:
top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
top_data_df['domain_name'] = top_data_df['domain'].str.split('.').str[-2]
airflow=top_data_df.query('domain_name=="airflow"')


In [116]:
top_data_df[top_data_df.domain_name=="airflow"]

,rank,domain,domain_name
